In [ ]:
import relevant.libraries

class Convolution:
  def __init__(self):
	"""
	Attributes for instance of EncoderDecoder module
	"""
	self.mod = self.getSourceModule()
	pass

  def getSourceModule(self):
	# kernel code wrapper
	kernelwrapper = """""" # you can either use a string or save the kernel in kernel.cu file and reference it here.
	# Compile the kernel code when an instance
	# of this class is made. 
	return SourceModule(kernelwrapper)

  def conv_gpu_naive(self):
	# implement this, note you can change the function signature (arguments and return type)
	pass

  def conv_gpu_shared_mem(self):
	# implement this, note you can change the function signature (arguments and return type)
	pass

  def conv_gpu_shared_and_constant_mem(self):
	# implement this, note you can change the function signature (arguments and return type)
	pass

  def test_conv_pycuda(self):
	# implement this, note you can change the function signature (arguments and return type)
	pass